# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F

In [4]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/SPH"
print(os.listdir(data_dir))

['TrainingSet3', 'Label.csv', 'alldata', 'TrainingSet1', 'single_label.csv', 'TrainingSet2']


In [5]:
main_df = pd.read_csv(data_dir + "/data_df.csv")
main_df.shape

(6877, 4)

In [6]:
single_fns = main_df["File name"].values.tolist()
single_mat_paths = [data_dir + f"/records/{x}.h5" for x in single_fns]

In [7]:
# ratio = [0.9, 0.1, 0.1]

# train_index = int(len(single_mat_paths)*ratio[0])

# train_mat_paths = single_mat_paths[:train_index]
# valid_mat_paths = single_mat_paths[train_index:]

# train_label = main_df.iloc[:train_index,:]
# valid_label = main_df.iloc[train_index:,:]

ratio = [0.8, 0.1]

train_index = int(len(single_mat_paths)*ratio[0])
valid_index = int(len(single_mat_paths)*(ratio[0]+ratio[1]))

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[train_index:valid_index]
# test_mat_paths = single_mat_paths[valid_index:]

train_label = main_df.iloc[:train_index,:]
valid_label = main_df.iloc[train_index:valid_index,:]
# test_label = single_main_df.iloc[valid_index:,:]

 # Data Loader

In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot
import h5py

In [9]:
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]
        a = h5py.File(data_path, 'r')
        data_h5 = a['ecg']
        data = np.array(data_h5)
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        grayscale_images = (normalized_data * 255)
        grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        resized_images = F.interpolate(grayscale_images, size=(9*4,2500), mode='bilinear', align_corners=True)
        resized_images = resized_images.squeeze(0).squeeze(0)
        torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        torch_data_resize = self.transform(torch_data)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["File name"] == filename]["New Label"].values.item()

        return torch_data_resize, label

    def __len__(self):
        return len(self.data_paths) 

In [10]:
train_ds = HeartData(train_mat_paths, main_df)
valid_ds = HeartData(valid_mat_paths, main_df)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [12]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.ori_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 9),
            nn.Softmax(dim = 1)
        )
        
    def forward(self, x):        
        x = self.ori_model(x)
        return x

model = HeartModel()
# x = torch.randn((1, 3, 256, 512)).to("cuda")
# out = model(x)
# print(out.shape)

In [13]:
model

HeartModel(
  (ori_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Training

In [14]:
epoch = 150
lr = 0.001 
best_acc = 0
best_ep = 0

# 0.05 Acc: 0.241 F1-score: 0.080 
# 0.01 Acc: 0.241 0.238 F1-score: 0.094 0.109

class_la = [1,2,3,4,5,6,7,8,9]
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])

model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))
loss_fn = nn.CrossEntropyLoss()

In [15]:
class CBFocalClassifierV0(nn.Module):
    def __init__(self, gamma = 2):
        super(CBFocalClassifierV0, self).__init__()
        
        self.gamma = gamma
    
    def forward(self, pred, target):

        cls_loss = {}

        logits = F.log_softmax(pred, dim=1)

        B = list(target.size())[0]
        
        beta = (B - 1)/B
        
        for b_logits, b_target in zip(logits, target):
            target_idx = b_target.item()
            entropy = torch.sum(torch.pow(1 - b_logits.exp(), self.gamma) * b_logits * b_target)
            if target_idx in cls_loss:
                cls_loss[target_idx].append(entropy)
            else:
                cls_loss[target_idx] = [entropy]
        
        sum_cls_loss = {
            _cls : ((1 - beta)/(1 - beta ** len(cls_loss[_cls]) + 1e-6)) * sum(cls_loss[_cls]) for _cls in cls_loss
        }

        return (-1 / B) * sum(list(sum_cls_loss.values()))

focalloss_fn = CBFocalClassifierV0()

In [16]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    y_true_list = [] 
    pred_list = []
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            
            pred_pos = pred.argmax(1)
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        
y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

reports = classification_report(y_true, pred, target_names=class_la, output_dict=True) 

print(reports)
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

Epoch: 0


80it [00:11,  7.17it/s]

train loss: 8.896227051940146 - train acc: 11.640625



640it [00:04, 158.67it/s]

valid loss: 2.2009341821237722 - valid acc: 7.968749999999999
Epoch: 1


...

valid loss: 2.200661608497488 - valid acc: 14.374999999999998
Epoch: 149



80it [00:14,  5.44it/s]

train loss: 8.825593030905422 - train acc: 10.80078125



640it [00:05, 108.86it/s]

valid loss: 2.2006614674611455 - valid acc: 13.4375
{'1': {'precision': 0.09836065573770492, 'recall': 0.06451612903225806, 'f1-score': 0.07792207792207792, 'support': 93.0}, '2': {'precision': 0.1813186813186813, 'recall': 0.3235294117647059, 'f1-score': 0.2323943661971831, 'support': 102.0}, '3': {'precision': 0.10126582278481013, 'recall': 0.12698412698412698, 'f1-score': 0.11267605633802817, 'support': 63.0}, '4': {'precision': 0.043478260869565216, 'recall': 0.07692307692307693, 'f1-score': 0.05555555555555555, 'support': 13.0}, '5': {'precision': 0.24324324324324326, 'recall': 0.11688311688311688, 'f1-score': 0.15789473684210528, 'support': 154.0}, '6': {'precision': 0.08823529411764706, 'recall': 0.10526315789473684, 'f1-score': 0.09599999999999999, 'support': 57.0}, '7': {'precision': 0.16216216216216217, 'recall': 0.1, 'f1-score': 0.12371134020618559, 'support': 60.0}, '8': {'precision': 0.11666666666666667, 'recall': 0.07865168539325842, 'f1-score': 0.09395973154362416, 'supp

# Evaluation